<a href="https://colab.research.google.com/github/saugabriele/Machine_Learning_Project/blob/main/Machine_Learning_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Malicious Url Detector Project**
To develop a classifier for detecting malicious URLs, the project started with a dataset containing a few URLs. In this case I tried to create a multiclass classifier that can classify a URL like:
- **Benign**: These are safe to browse URLs.
- **Phishing**: By creating phishing URLs, hackers try to steal sensitive personal or financial information such as login credentials
- **Malware**:These type of URLs inject malware into the victim’s system once he/she visit such URLs.
- **Defacement**: Defacement URLs are generally created by hackers with the intention of breaking into a web server and replacing the hosted website with one of their own

In [1]:
import pandas as pd
import numpy as np
import re
from urllib.parse import urlparse

In [2]:
#Create a DataFrame from the dataset csv file
df = pd.read_csv("https://raw.githubusercontent.com/saugabriele/Machine_Learning_Project/main/malicious_phish1.csv")
df.head()

,url,type
0,br-icloud.com.br,phishing
1,mp3raid.com/music/krizz_kaliko.html,benign
2,bopsecrets.org/rexroth/cr/1.htm,benign
3,http://www.garage-pirenne.be/index.php?option=...,defacement
4,http://adventure-nicaragua.net/index.php?optio...,defacement


In [3]:
#Convert the DataFrame in numpy arrays
url_array = df['url'].to_numpy()
type_array = df['type'].to_numpy()

##**Features Extraction**
To determine the class label of an URL, the first step was selecting the features that can be extracted from an URL. In this case i decided to extract some lexical features:
- These refer to statistical features extracted from the literal URL string. For example, length of the URL string, number of digits, number of parameters in its query part, if the URL is encoded, etc.

In [4]:
def www_count(url):
  """
    From the input URL this will return the number of 
    occurrences of the substring 'www'.
  """
  return url.count('www')

def at_count(url):
  """
    From the input URL this will return the number of 
    occurrences of the substring '@'.
  """
  return url.count('@')

def url_lenght(url):
  """
    From the input URL this will return the
    corresponding length
  """
  return len(url)

def url_path_len(url):
  """
    From the input URL this will return the length
    of the path where the resource is located.
  """
  url_parse = urlparse(url)
  return len(url_parse.path)

def host_len(url):
  """
     From the input URL this will return the length
     of the hostname.
  """
  url_parse = urlparse(url)
  return len(url_parse.netloc)

def url_host_is_ip(url):
  """
    From the input URL this will return 1 if the hostname
    contains the IP adress otherwise it returns 0 if it
    contains the domain name where the resource is located.
  """
  url = urlparse(url)
  reg = r"^(([0-9]|[1-9][0-9]|1[0-9]{2}|2[0-4][0-9]|25[0-5])\.){3}([0-9]|[1-9][0-9]|1[0-9]{2}|2[0-4][0-9]|25[0-5])$"
  if re.match(reg, url.netloc):
      return 1
  else:
      return 0

def number_of_digits(url):
  """
    From the input URL this will return the number of 
    digits in the URL.
  """
  digits = [i for i in url if i.isdigit()]
  return len(digits)

def number_of_parameters(url):
  """
    From the input URL this will return the number of
    parameters contained in the URL.
  """
  params = urlparse(url).query
  return 0 if params == '' else len(params.split('&'))

def number_of_subdirectories(url):
  """
    From the input URL this will return the number of
    subdirectories in the path of the URL.
  """
  d = urlparse(url).path.split('/')
  return len(d) - 1

def number_of_periods(url):
  """
    From the input URL this will return the number of
    periods in the URL.
  """
  return url.count('.')

def num_encoded_char(url):
  """
    From the input URL this will return the number of
    encoded characters in the URL.
  """
  return url.count('%')

def equal_count(url):
  """
    From the input URL this will return the number of
    variable values passed from one form page to another.
  """
  return url.count('=')

def count_http(url):
  """
    From the input URL this will return the number of 
    occurrences of the substring 'http'.
  """
  return url.count('http')

def count_https(url):
  """
    From the input URL this will return the number of 
    occurrences of the substring 'https'.
  """
  return url.count('https')

features_extraction_func = []
features_extraction_func.append(np.vectorize(www_count))
features_extraction_func.append(np.vectorize(at_count))
features_extraction_func.append(np.vectorize(url_lenght))
features_extraction_func.append(np.vectorize(url_path_len))
features_extraction_func.append(np.vectorize(host_len))
features_extraction_func.append(np.vectorize(url_host_is_ip))
features_extraction_func.append(np.vectorize(number_of_digits))
features_extraction_func.append(np.vectorize(number_of_parameters))
features_extraction_func.append(np.vectorize(number_of_subdirectories))
features_extraction_func.append(np.vectorize(number_of_periods))
features_extraction_func.append(np.vectorize(num_encoded_char))
features_extraction_func.append(np.vectorize(equal_count))
features_extraction_func.append(np.vectorize(count_http))
features_extraction_func.append(np.vectorize(count_https))

In [5]:
def make_predictor_variables(features_extraction_func, urls):
  x = np.zeros(shape = (url_array.shape[0], len(features_extraction_func)))

  for i, function in enumerate(features_extraction_func):
    x[:, i] = function(urls)

  return x

x = make_predictor_variables(features_extraction_func, url_array)